In [10]:
%load_ext autoreload
%autoreload 2
import pandas as pd
from ai_cdss.data_loader import DataLoader
from ai_cdss.data_processor import DataProcessor

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
rgs_mode = "app"
scoring_weights = [1,1,1]
ewma_alpha = 0.5

In [12]:
patient_list = [
    775,  787,  788, 1123, 1169, 1170, 1171, 1172, 1173, 1983, 2110, 2195,
    2955, 2956, 2957, 2958, 2959, 2960, 2961, 2962, 2963, 3081, 3229, 3318, 3432
]

In [13]:
loader = DataLoader(rgs_mode=rgs_mode)
processor = DataProcessor(weights=scoring_weights, alpha=ewma_alpha)

In [14]:
session = loader.load_session_data(patient_list=patient_list)
timeseries = loader.load_timeseries_data(patient_list=patient_list)
ppf = loader.load_ppf_data(patient_list=patient_list)

Database engine created successfully


INFO:ai_cdss.data_loader:Session data loaded successfully.


Database engine closed
Database engine created successfully
Database engine closed
Database engine created successfully


INFO:ai_cdss.data_loader:PPF data loaded successfully.


Database engine closed


In [15]:
scoring = processor.process_data(session_data=session, timeseries_data=timeseries, ppf_data=ppf)

#### CDSS

In [16]:
from ai_cdss.cdss import CDSS

In [17]:
protocol_similarity = loader.load_protocol_similarity()

INFO:ai_cdss.data_loader:Protocol similarity data loaded successfully.


In [18]:
cdss = CDSS(
    scoring=scoring
)

In [19]:
patient_id = 775
protocol_id = 227

In [20]:
score_row = scoring[
    (scoring["PATIENT_ID"] == patient_id) & (scoring["PROTOCOL_ID"] == protocol_id)
]

In [21]:
for patient_id in patient_list:
    recommendation = cdss.recommend(patient_id=patient_id, protocol_similarity=protocol_similarity)
    break

In [22]:
cdss.get_prescriptions(patient_id)

,PATIENT_ID,PROTOCOL_ID,PPF,CONTRIB,ADHERENCE,DM_VALUE,PE_VALUE,USAGE,DAYS,SCORE
38,775,206,0.528646,"[0.013972368519644753, 0.13972368519644754, 0....",0.770777,1.000000,0.832768,87,"[0, 4]",2.299424
33,775,208,0.484665,"[0.02088597029033065, 0.13923980193553767, 0.0...",0.674691,0.994970,0.974205,242,"[0, 2, 4]",2.154326
31,775,214,0.617411,"[0.014909525935815035, 0.04969841978605012, 0....",0.609462,0.949463,0.934906,142,[2],2.176337
45,775,222,0.632038,"[0.022302696143133578, 0.07434232047711194, 0....",0.985924,1.000000,1.000000,225,"[0, 4]",2.617962
46,775,224,0.632038,"[0.022302696143133578, 0.07434232047711194, 0....",1.000000,0.672785,0.709470,95,"[0, 4]",2.304824
48,775,227,0.097419,"[0.03404920730373373, 0.0, 0.0, 0.0, 0.0, 0.01...",1.000000,0.000000,0.000000,4,[2],1.097419


In [23]:
recommendation.to_parquet("recommendation.parquet")

In [38]:
list(recommendation['DAYS'].explode())

[0, 4, 0, 2, 4, 2, 0, 4, 0, 4, 2]

In [40]:
from collections import Counter

In [42]:
# Flatten all day assignments for this patient
collapsed_days_list = list(recommendation['DAYS'].explode())

# Count how many times each day appears
day_counts = Counter(collapsed_days_list)

In [43]:
day_counts

Counter({0: 4, 4: 4, 2: 3})

In [ ]:
# ✅ Assert each day (0–6) appears exactly 2 times
for day in range(7):
    assert day_counts[day] == 2, f"Day {day} has {day_counts[day]} prescriptions (expected 2)"

AssertionError: Day 0 has 4 prescriptions (expected 2)

In [46]:
import numpy as np
set(np.arange(0,7,1))

{0, 1, 2, 3, 4, 5, 6}

In [33]:
for row in recommendation['DAYS']:
    print(row)

[0, 4]
[0, 2, 4]
[2]
[0, 4]
[0, 4]
[2]
